# The Zimnat Insurance Assurance Challenge by #ZindiWeekendz - Farzi Data Scientists (Rank 3)

*The data describes 51,685 life assurance policies, each identified by a unique Policy ID. Each year, some policies lapse as clients change jobs, move countries etc. The information on these policies are in multiple files.*

*Numerical quantities have been transformed, and many categories have been assigned unique IDs in place of the original text.*

*The objective of this hackathon is to develop a predictive model that determines the likelihood for a customer to churn - to seek an alternative insurer or simply stop paying for insurance altogether.*

*sample_submission.csv contains rows for the remaining Policy IDs (with ‘?’s in TRAIN). You must predict which of these policies marked with '?' will lapse in 2020*

* client_data.csv - Contains some personal information on the principal member, such as location, branch and agent code, age etc.
* payment_history.csv - Contains payment history up to the end of 2018 tied to Policy ID. Payments made in 2019 are not provided.
* policy_data.csv - Describes the policies themselves. There may be multiple rows for each Policy ID since policies can cover more than one person.
* train.csv - contains a list of all the policies. Policies that lapsed in 2017, 2018 or 2019 are identified with a 1 in the ‘Lapse’ column, and the year is provided. The policies with a '?' in the 'Lapse' and 'Lapse Year' column are the policies that remained and had not lapsed as of the end of 2019. You must estimate the likelihood that these policies lapsed or not in 2020.
* sample_submission.csv - is an example of what your submission should look like. The order of the rows does not matter but the name of the ID must be correct.
* variable_defintions.txt - definitions of the variables

In [99]:
from lightgbm import LGBMClassifier
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from tqdm import *
from sklearn.metrics import *
import warnings 
warnings.simplefilter('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactive = "all"

In [101]:
%%time
payment_history = pd.read_csv('payment_history.csv')
client_data = pd.read_csv('client_data.csv')
policy_data = pd.read_csv('policy_data.csv')
train = pd.read_csv('train.csv')
sample_sub = pd.read_csv('sample_sub.csv')

CPU times: user 1.85 s, sys: 36 ms, total: 1.89 s
Wall time: 1.88 s


Our final solution used only **policy_data**.
**client_data** cannot be used since it will not vary with time. (Or maybe it did, and we did not find out)
**payment_history** was given only till 2018, and caused overfitting for 2020 test data, hence we ignored it in our final solution.

In [102]:
policy_data.shape, payment_history.shape

((282815, 14), (495503, 5))

In [103]:
policy_data = policy_data.drop_duplicates()
payment_history = payment_history.drop_duplicates()

In [104]:
policy_data.shape, payment_history.shape

((278988, 14), (482179, 5))

In [105]:
policy_data.nunique()

Policy ID          51685
NP2_EFFECTDATE        43
PPR_PRODCD            17
NPR_PREMIUM         2234
NPH_LASTNAME       25275
CLF_LIFECD             6
NSP_SUBPROPOSAL      171
NPR_SUMASSURED      1200
NLO_TYPE               6
NLO_AMOUNT           974
AAG_AGCODE           591
PCL_LOCATCODE         15
OCCUPATION           240
CATEGORY               6
dtype: int64

In [106]:
policy_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278988 entries, 0 to 282814
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Policy ID        278988 non-null  object 
 1   NP2_EFFECTDATE   278988 non-null  object 
 2   PPR_PRODCD       278988 non-null  object 
 3   NPR_PREMIUM      278934 non-null  float64
 4   NPH_LASTNAME     278988 non-null  object 
 5   CLF_LIFECD       278988 non-null  int64  
 6   NSP_SUBPROPOSAL  278988 non-null  int64  
 7   NPR_SUMASSURED   185357 non-null  float64
 8   NLO_TYPE         278988 non-null  object 
 9   NLO_AMOUNT       88654 non-null   float64
 10  AAG_AGCODE       278988 non-null  object 
 11  PCL_LOCATCODE    278988 non-null  object 
 12  OCCUPATION       278988 non-null  object 
 13  CATEGORY         278988 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 31.9+ MB


In [107]:
policy_data.head()

,Policy ID,NP2_EFFECTDATE,PPR_PRODCD,NPR_PREMIUM,NPH_LASTNAME,CLF_LIFECD,NSP_SUBPROPOSAL,NPR_SUMASSURED,NLO_TYPE,NLO_AMOUNT,AAG_AGCODE,PCL_LOCATCODE,OCCUPATION,CATEGORY
0,PID_EPZDSP8,1/9/2019,PPR_PRODCD_B2KVCE7,265.724174,NPH_LASTNAME_BPN2LEB,2,222,NaN,NLO_TYPE_DPBHSAH,NaN,AAG_AGCODE_APWOOPE,PCL_LOCATCODE_7SHK7I9,OCCUPATION_NNHJ7XV,CATEGORY_GWW4FYB
1,PID_6M6G9IB,1/8/2018,PPR_PRODCD_64QNIHM,2795.069380,NPH_LASTNAME_U2H3GC6,1,111,213380.713197,NLO_TYPE_XTHV3A3,609.054794,AAG_AGCODE_9Z3FBGA,PCL_LOCATCODE_7VFS3EQ,OCCUPATION_IKCIDKW,CATEGORY_R821UZV
2,PID_UL0F7LH,1/8/2017,PPR_PRODCD_KOFUYNN,2492.759107,NPH_LASTNAME_B68RERV,1,111,238857.872515,NLO_TYPE_XAJI0Y6,1339.461987,AAG_AGCODE_Y0LKFF0,PCL_LOCATCODE_SKPRCR4,OCCUPATION_NUJZA7T,CATEGORY_8DALFYO
3,PID_TRGUBTU,1/4/2018,PPR_PRODCD_KOFUYNN,3982.538095,NPH_LASTNAME_NPN3VGI,1,111,74968.903115,NLO_TYPE_XAJI0Y6,7870.961557,AAG_AGCODE_1OCF2N0,PCL_LOCATCODE_SPQHMX5,OCCUPATION_W9XA3KX,CATEGORY_LXSLG6M
4,PID_TODLPIB,1/12/2019,PPR_PRODCD_KOFUYNN,1143.953733,NPH_LASTNAME_9VSNH0E,3,555,238857.872515,NLO_TYPE_DPBHSAH,NaN,AAG_AGCODE_E31VV8B,PCL_LOCATCODE_0T6GYGX,OCCUPATION_NNHJ7XV,CATEGORY_GWW4FYB


In [108]:
policy_data['NP2_EFFECTDATE'] = pd.to_datetime(policy_data['NP2_EFFECTDATE'], format = '%d/%m/%Y')
obj_cols = [c for c in policy_data.select_dtypes('object').columns.tolist() if c != 'Policy ID']
policy_data[obj_cols] = policy_data[obj_cols].apply(lambda x: pd.factorize(x)[0])

policy_data['NPR_PREMIUM - NLO_AMOUNT'] = policy_data['NPR_PREMIUM'] - policy_data['NLO_AMOUNT']
policy_data['NPR_PREMIUM / NPR_SUMASSURED'] = policy_data['NPR_PREMIUM'] / policy_data['NPR_SUMASSURED']

def get_last_payment_diff(x):
    try:
        return (x['NP2_EFFECTDATE'].values[-1] - x['NP2_EFFECTDATE'].values[-2])/ np.timedelta64(1, 'D')
    except:
        return np.nan

def get_pd_agg(pd):
    pd['policy_count'] = pd['Policy ID'].map(pd['Policy ID'].value_counts())
    aggs = {'NP2_EFFECTDATE': ['min', 'max', 'nunique', 'size'],
           'NPR_PREMIUM': ['mean', 'min', 'max', 'sum', 'std', 'nunique'],
           'NPR_SUMASSURED': ['mean','min', 'max', 'sum'],
            'NLO_AMOUNT': ['mean', 'min', 'max', 'sum', 'std', 'nunique'],
           'policy_count': ['sum', 'mean', 'std',],
           'NPR_PREMIUM - NLO_AMOUNT': ['sum', 'mean', 'std'],
           'NPR_PREMIUM / NPR_SUMASSURED': ['mean', 'std',]}
    pd_agg = pd.groupby('Policy ID').agg(aggs)
    pd_agg.columns = ['_'.join(c).strip('_') for c in pd_agg.columns]
    return pd_agg

In [109]:
policy_data.columns

Index(['Policy ID', 'NP2_EFFECTDATE', 'PPR_PRODCD', 'NPR_PREMIUM',
       'NPH_LASTNAME', 'CLF_LIFECD', 'NSP_SUBPROPOSAL', 'NPR_SUMASSURED',
       'NLO_TYPE', 'NLO_AMOUNT', 'AAG_AGCODE', 'PCL_LOCATCODE', 'OCCUPATION',
       'CATEGORY', 'NPR_PREMIUM - NLO_AMOUNT', 'NPR_PREMIUM / NPR_SUMASSURED'],
      dtype='object')

In [110]:
def get_features(df, year = 2020):
    '''
    This function calculates the stats and generates new features only upto the year specified.
    e.g: year = 2018 implies years 2019 and 2020 are ignored while creating new features
    '''
    
    ### Policy data features
    pdata = policy_data[policy_data['NP2_EFFECTDATE'].dt.year <= year]
    
    ### Filter the policy data so it contains data only upto the previous year
    prev_pdata = policy_data[policy_data['NP2_EFFECTDATE'].dt.year <= year-1]
    
    ### Get aggregate features upto current year
    pd_agg = get_pd_agg(pdata)
    df = pd.merge(left=df, right=pd_agg, on = 'Policy ID', how = 'left')
    
    ### Change in sum of numerical features between current year and previous year
    for c in ['NPR_SUMASSURED', 'NPR_PREMIUM', 'NLO_AMOUNT']:
        t1 = pdata.groupby('Policy ID')[c].sum()
        t2 = prev_pdata.groupby('Policy ID')[c].sum()
        t_diff = (t1 - t2).reset_index().rename({c: f'{c}_sum_change'}, axis=1).fillna(0)
        df = pd.merge(df, t_diff, on = 'Policy ID', how = 'left')
    
    
    ### Change and ratio of policy counts between current year and Previous year
    policy_cnt_curr_yr = pdata['Policy ID'].value_counts()
    policy_cnt_prev_yr = prev_pdata['Policy ID'].value_counts()
    
    policy_cnt_diff = (policy_cnt_curr_yr - policy_cnt_prev_yr).reset_index().rename({'index': 'Policy ID', 'Policy ID': 'policy_count_change'}, axis=1).fillna(0)
    df = pd.merge(df, policy_cnt_diff, on = 'Policy ID', how = 'left')
    
    policy_cnt_change_ratio = (policy_cnt_curr_yr / policy_cnt_prev_yr).reset_index().rename({'index': 'Policy ID', 'Policy ID': 'policy_count_change_ratio'}, axis=1).fillna(0)
    df = pd.merge(df, policy_cnt_change_ratio, on = 'Policy ID', how = 'left')
    
    ### Change in mean of numerical features between current year and previous year
    for c in ['NPR_PREMIUM', 'NPR_SUMASSURED','NLO_AMOUNT']:
        curr = pdata.groupby('Policy ID')[c].mean()
        prev = prev_pdata.groupby('Policy ID')[c].mean()
        diff = (curr - prev).reset_index().rename({'index': 'Policy ID', c: f'{c}_change'}, axis=1).fillna(0)
        df = pd.merge(df, diff, on = 'Policy ID', how = 'left')
    
    return df

In [111]:
### Train using only the data upto 2018. Data upto 2018 implies any policies that have lapsed in 2019 and 2020, haven't lapsed yet (We are still in 2018 :P), so are 0.

trn_2k18 = train.copy()
trn_2k18['target'] = 1
trn_2k18.loc[~trn_2k18['Lapse Year'].isin(['2017', '2018']), 'target'] = 0
trn_2k18 = get_features(trn_2k18, year = 2018)

In [112]:
### Validate using 2019's data
val_2k19 = train[~train['Lapse Year'].isin(['2017', '2018'])]
val_2k19['target'] = 1
val_2k19.loc[val_2k19['Lapse Year'] == '?', 'target'] = 0
val_2k19 = get_features(val_2k19, year = 2019)

#### Train again using complete data, and predict on test data
trn_all = train.copy()
trn_all['target'] = 1
trn_all.loc[trn_all['Lapse Year'] == '?', 'target'] = 0
trn_all = get_features(trn_all, year = 2020)

test = train[train['Lapse Year'] == '?']
test = pd.merge(sample_sub[['Policy ID']], test, on = 'Policy ID', how = 'left')
test = get_features(test, year = 2020)

test.shape, sample_sub.shape

((43707, 39), (43707, 2))

In [113]:
### Highly imbalanced dataset.
trn_2k18['target'].value_counts(normalize=True)

0    0.942517
1    0.057483
Name: target, dtype: float64

In [114]:
### Highly imbalanced dataset.
trn_all['target'].value_counts(normalize=True)

0    0.845642
1    0.154358
Name: target, dtype: float64

In [115]:
### Try setting all the submission values, to this constant value, you will get 0.30xxx as score, without doing anything. That was a baseline :D
### 0.057 -> 1 count for year upto 2018
### 0.154 -> 1 count for year upto 2019

(0.057 + 0.154)/2

0.1055

In [116]:
### Some features were overfitting hence ignored

features = [c for c in trn_all.columns if c not in ['Policy ID', 'Lapse', 'Lapse Year', 'NP2_EFFECTDATE_min', 'NP2_EFFECTDATE_max', 'target']]
features

['NP2_EFFECTDATE_nunique',
 'NP2_EFFECTDATE_size',
 'NPR_PREMIUM_mean',
 'NPR_PREMIUM_min',
 'NPR_PREMIUM_max',
 'NPR_PREMIUM_sum',
 'NPR_PREMIUM_std',
 'NPR_PREMIUM_nunique',
 'NPR_SUMASSURED_mean',
 'NPR_SUMASSURED_min',
 'NPR_SUMASSURED_max',
 'NPR_SUMASSURED_sum',
 'NLO_AMOUNT_mean',
 'NLO_AMOUNT_min',
 'NLO_AMOUNT_max',
 'NLO_AMOUNT_sum',
 'NLO_AMOUNT_std',
 'NLO_AMOUNT_nunique',
 'policy_count_sum',
 'policy_count_mean',
 'policy_count_std',
 'NPR_PREMIUM - NLO_AMOUNT_sum',
 'NPR_PREMIUM - NLO_AMOUNT_mean',
 'NPR_PREMIUM - NLO_AMOUNT_std',
 'NPR_PREMIUM / NPR_SUMASSURED_mean',
 'NPR_PREMIUM / NPR_SUMASSURED_std',
 'NPR_SUMASSURED_sum_change',
 'NPR_PREMIUM_sum_change',
 'NLO_AMOUNT_sum_change',
 'policy_count_change',
 'policy_count_change_ratio',
 'NPR_PREMIUM_change',
 'NPR_SUMASSURED_change',
 'NLO_AMOUNT_change']

In [117]:
trn_all.head()

,Policy ID,Lapse,Lapse Year,target,NP2_EFFECTDATE_min,NP2_EFFECTDATE_max,NP2_EFFECTDATE_nunique,NP2_EFFECTDATE_size,NPR_PREMIUM_mean,NPR_PREMIUM_min,...,NPR_PREMIUM / NPR_SUMASSURED_mean,NPR_PREMIUM / NPR_SUMASSURED_std,NPR_SUMASSURED_sum_change,NPR_PREMIUM_sum_change,NLO_AMOUNT_sum_change,policy_count_change,policy_count_change_ratio,NPR_PREMIUM_change,NPR_SUMASSURED_change,NLO_AMOUNT_change
0,PID_4928TWH,?,?,0,2017-08-01,2017-08-01,1,1,42911.077278,42911.077278,...,0.125962,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,PID_KBLLEGK,?,?,0,2018-05-01,2018-05-01,1,2,3561.268991,3561.268991,...,0.021576,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,PID_90F0QA3,?,?,0,2019-09-01,2019-09-01,1,2,6164.812836,6164.812836,...,0.024418,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,PID_18F3NHF,?,?,0,2019-12-01,2019-12-01,1,1,2278.189789,2278.189789,...,0.009538,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,PID_SX4QUVO,?,?,0,2019-03-01,2019-03-01,1,1,1619.046308,1619.046308,...,0.011165,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [118]:
### Some features can be removed still

drop_features = [c for c in features if trn_all[c].sum() == 0]
features = [c for c in features if c not in drop_features]
print(drop_features)

['policy_count_std']


In [119]:
### Some features were overfitting hence ignored

features = [c for c in features if c not in ['AMOUNTPAID', 'NP2_EFFECTDATE_days_diff', 'policy_count_std']]

## Training on 2018, validation on 2019

In [120]:
clf = LGBMClassifier(num_leaves=32,max_depth=18, learning_rate=0.01, reg_alpha=1,reg_lambda=1, n_estimators=600, subsample=1, subsample_freq=1, colsample_bytree=0.55)
clf.fit(trn_2k18[features], trn_2k18['target'], eval_set = [(val_2k19[features], val_2k19['target'])], verbose = 20)

[20]	valid_0's binary_logloss: 0.317732
[40]	valid_0's binary_logloss: 0.300291
[60]	valid_0's binary_logloss: 0.291181
[80]	valid_0's binary_logloss: 0.283651
[100]	valid_0's binary_logloss: 0.278971
[120]	valid_0's binary_logloss: 0.275966
[140]	valid_0's binary_logloss: 0.273309
[160]	valid_0's binary_logloss: 0.271762
[180]	valid_0's binary_logloss: 0.270549
[200]	valid_0's binary_logloss: 0.269712
[220]	valid_0's binary_logloss: 0.269263
[240]	valid_0's binary_logloss: 0.268986
[260]	valid_0's binary_logloss: 0.268713
[280]	valid_0's binary_logloss: 0.268712
[300]	valid_0's binary_logloss: 0.268737
[320]	valid_0's binary_logloss: 0.268734
[340]	valid_0's binary_logloss: 0.268875
[360]	valid_0's binary_logloss: 0.269038
[380]	valid_0's binary_logloss: 0.269172
[400]	valid_0's binary_logloss: 0.269256
[420]	valid_0's binary_logloss: 0.269424
[440]	valid_0's binary_logloss: 0.269646
[460]	valid_0's binary_logloss: 0.269832
[480]	valid_0's binary_logloss: 0.270026
[500]	valid_0's bina

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.55,
               importance_type='split', learning_rate=0.01, max_depth=18,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=600, n_jobs=-1, num_leaves=32, objective=None,
               random_state=None, reg_alpha=1, reg_lambda=1, silent=True,
               subsample=1, subsample_for_bin=200000, subsample_freq=1)

## Training on complete data.

* The number of iterations have been increased a little more compared to the best iteration for validation

In [121]:
tp = pd.DataFrame()
for i in tqdm_notebook(range(7)):
    clf = LGBMClassifier(num_leaves=32, learning_rate=0.01, reg_alpha=2, n_estimators=380, subsample=1, subsample_freq=1, colsample_bytree=0.5, random_state=2**i)
    clf.fit(trn_all[features], trn_all['target'], eval_set = [(val_2k19[features], val_2k19['target'])], verbose = 380)
    tp[i] = clf.predict_proba(test[features])[:, 1]
test_preds = tp.mean(axis=1)

[380]	valid_0's binary_logloss: 0.252244
[380]	valid_0's binary_logloss: 0.251058
[380]	valid_0's binary_logloss: 0.251556
[380]	valid_0's binary_logloss: 0.251686
[380]	valid_0's binary_logloss: 0.251647
[380]	valid_0's binary_logloss: 0.251662
[380]	valid_0's binary_logloss: 0.251376



Lets see some stats about our target

In [122]:
test_preds = tp.mean(axis=1)
print(pd.Series(test_preds).describe())

count    43707.000000
mean         0.128244
std          0.130293
min          0.003687
25%          0.003703
50%          0.104520
75%          0.234366
max          0.653318
dtype: float64


This thresholding greatly improved our score, from 0.260x to 0.24xx :D. The intuition is log loss heavily penalizes confident wrong predictions. Also most of the values were 0, so using a threshold to rectify it, seemed ok.

In [123]:
test_preds[test_preds < 0.03] = 0
test_preds[test_preds > 0.18] = 0.18
pd.Series(test_preds).describe()

count    43707.000000
mean         0.090678
std          0.086338
min          0.000000
25%          0.000000
50%          0.104520
75%          0.180000
max          0.180000
dtype: float64

In [124]:
SUB_FILE_NAME = 'all_preds.csv'
sample_sub['Lapse'] = test_preds
sample_sub.to_csv(SUB_FILE_NAME, index=False)

from IPython.display import HTML
def create_download_link(title = "Download CSV file", filename = "data.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)
create_download_link(filename = SUB_FILE_NAME)